<a href="https://colab.research.google.com/github/andrewhywong/SDM-GeoAttribution/blob/main/analysis_global/global_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialize Earth Engine

In [ ]:
import time
import ee
import folium

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=OeoeasauUaoAIkReAU7-BQxCDCx9XXbbygxUDlEEqVU&tc=ptkEA4OI5SXM5DgKzupOo4Ky2zI-qvGrmFEOPipt0mM&cc=jjYqkwN-FAjW4O-YsmAz3wXcIgWnwbtq6KUV2COt7RA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh4ECQTnfLgxZcy3eQiCwggWzeGYeaKWEkvBUHaU1keXR4uO-5vBp9A

Successfully saved authorization token.


## Define functions for modeling

In [ ]:
# check asset list and make sure it is updated
def get_asset_list(parent):
    parent_asset = ee.data.getAsset(parent)
    parent_id = parent_asset['name']
    parent_type = parent_asset['type']
    asset_list = []
    child_assets = ee.data.listAssets({'parent': parent_id})['assets']
    for child_asset in child_assets:
        child_id = child_asset['name']
        child_type = child_asset['type']
        if child_type in ['FOLDER','IMAGE_COLLECTION']:
            asset_list.extend(get_asset_list(child_id))
        else:
            asset_list.append(child_id)
    return asset_list

# remove duplicated occurrences in the same pixel
def RemoveDuplicates(data):
  randomraster = ee.Image.random().reproject('EPSG:4326', None, GrainSize)
  randpointvals = randomraster.sampleRegions(collection=ee.FeatureCollection(data), scale=10,geometries=True)
  return randpointvals.distinct('random')

# set precenses and absences
def setPresence(feature):
    return feature.set('PresAbs', 1)
def setAbsence(feature):
    return feature.set('PresAbs', 0)

# Create grids for spatial block CV 
def makeGrid(geometry, scale):
  lonLat = ee.Image.pixelLonLat();
  lonGrid = lonLat.select('longitude').multiply(100000).toInt();
  latGrid = lonLat.select('latitude').multiply(100000).toInt();
  return lonGrid.multiply(latGrid).reduceToVectors(
      geometry=geometry,
      scale=scale,
      geometryType='polygon',
    );

# fit SDM models
def SDM(x):
    Seed = ee.Number(x);
    # Randomly split blocks for training and validation
    GRID = ee.FeatureCollection(Grid).randomColumn(seed=Seed).sort('random');
    TrainingGrid = GRID.filter(ee.Filter.lt('random', split));  
    TestingGrid = GRID.filter(ee.Filter.gte('random', split)); 

    # Presences
    PresencePoints = ee.FeatureCollection(Data);
    PresencePoints = PresencePoints.map(setPresence);
    TrPresencePoints = PresencePoints.filter(ee.Filter.bounds(TrainingGrid));  
    TePresencePoints = PresencePoints.filter(ee.Filter.bounds(TestingGrid));  
    
    # Pseudo-absences
    if pseudoAbs_mode == 'background':
      TrPseudoAbsPoints = AreaForPA.sample(region=TrainingGrid, scale=GrainSize, numPixels=50000, seed=Seed, geometries=True);
      TrPseudoAbsPoints = TrPseudoAbsPoints.map(setAbsence);
      TePseudoAbsPoints = AreaForPA.sample(region=TestingGrid, scale=GrainSize, numPixels=10000, seed=Seed, geometries=True); 
      TePseudoAbsPoints = TePseudoAbsPoints.map(setAbsence);

    elif pseudoAbs_mode == 'pseudoAbs_k_means':
      TrPseudoAbsPoints = AreaForPA.sample(region=TrainingGrid, scale=GrainSize, numPixels=TrPresencePoints.size().add(500), seed=Seed, geometries=True); # THIS SHOULD NOT IMPACT COMPUTATION TIMED OUT
      TrPseudoAbsPoints = TrPseudoAbsPoints.randomColumn().sort('random').limit(ee.Number(TrPresencePoints.size())); 
      TrPseudoAbsPoints = TrPseudoAbsPoints.map(setAbsence);
      
      TePseudoAbsPoints = AreaForPA.sample(region=TestingGrid, scale=GrainSize, numPixels=TePresencePoints.size().add(500), seed=Seed, geometries=True); 
      TePseudoAbsPoints = TePseudoAbsPoints.randomColumn().sort('random').limit(ee.Number(TePresencePoints.size())); 
      TePseudoAbsPoints = TePseudoAbsPoints.map(setAbsence);
    else:
      print("need to specify pseudo absence method!")

    trainingPartition = TrPresencePoints.merge(TrPseudoAbsPoints);
    testingPartition = TePresencePoints.merge(TePseudoAbsPoints);

    trainPixelVals = predictors.sampleRegions(collection=trainingPartition, properties=['PresAbs'], scale= GrainSize, tileScale= 16, geometries= True);

    if classifier_type == "Random_Forest":
      Classifier = ee.Classifier.smileRandomForest(
        numberOfTrees=500, #The number of decision trees to create.
        variablesPerSplit=None, #The number of variables per split. If unspecified, uses the square root of the number of variables.
        minLeafPopulation=10, #Only create nodes whose training set contains at least this many points. Integer, default: 1
        bagFraction=0.5, #The fraction of input to bag per tree. Default: 0.5.
        maxNodes=None, #The maximum number of leaf nodes in each tree. If unspecified, defaults to no limit.
        seed=Seed #The randomization seed.
        );
    elif classifier_type == "Gradient_Tree_Boost":
      Classifier = ee.Classifier.smileGradientTreeBoost(
        numberOfTrees=50, #The number of decision trees to create.
        shrinkage=0.001, #The shrinkage parameter in (0, 1) controls the learning rate of procedure. Default: 0.005
        samplingRate=0.75, #The sampling rate for stochastic tree boosting. Default 0.07
        maxNodes=None, #The maximum number of leaf nodes in each tree. If unspecified, defaults to no limit.
        loss="LeastAbsoluteDeviation", #Loss function for regression. One of: LeastSquares, LeastAbsoluteDeviation, Huber.
        seed=Seed #The randomization seed.
      );
    elif classifier_type == "MaxEnt":
      Classifier = ee.Classifier.amnhMaxent(
        autoFeature=True, #//Automatically select which feature classes to use, based on number of training samples.
        addSamplesToBackground=True, #//Add to the background any sample for which has a combination of environmental values that isn't already present in the background.
        extrapolate=True, #//Predict to regions of environmental space outside the limits encountered during training.
        seed=Seed #//The randomization seed.
      );
    else:
      print("No classifier_type specified!")
    
    # Get presence suitability
    ClassifierPr = Classifier.setOutputMode('PROBABILITY').train(trainPixelVals, 'PresAbs', bands); 
    ClassifiedImgPr = predictors.select(bands).classify(ClassifierPr);
    
    # # Binary presence/absence map
    # ClassifierBin = Classifier.setOutputMode('CLASSIFICATION').train(trainPixelVals, 'PresAbs', bands); 
    # ClassifiedImgBin = predictors.select(bands).classify(ClassifierBin);
   
    # return ee.List([ClassifiedImgPr, ClassifiedImgBin, trainingPartition, testingPartition]);
    return ee.List([ClassifiedImgPr, ClassifiedImgPr, trainingPartition, testingPartition]); # for maxent ClassifiedImgBin

# get result values
def getResults(x):
  return results.get(x)



## Define functions for accuracy assessments

In [ ]:
# accuracy assessment at different thresholds
def getAcc(img,TP):
  Pr_Prob_Vals = img.sampleRegions(collection=TP, properties=['PresAbs'], scale=GrainSize, tileScale= 16);
  seq = ee.List.sequence(start=0, end=1, count=25);
  def cut_(cutoff):
    Pres = Pr_Prob_Vals.filterMetadata('PresAbs','equals',1); #.filter(ee.filter.eq('PresAbs',1))
    # true-positive and true-positive rate, sensitivity  
    TP =  ee.Number(Pres.filterMetadata('classification','greater_than',cutoff).size());
    TPR = TP.divide(Pres.size());
    Abs = Pr_Prob_Vals.filterMetadata('PresAbs','equals',0);
    # false-negative
    FN = ee.Number(Pres.filterMetadata('classification','less_than',cutoff).size());
    # true-negative and true-negative rate, specificity  
    TN = ee.Number(Abs.filterMetadata('classification','less_than',cutoff).size());
    TNR = TN.divide(Abs.size());
    # false-positive and false-positive rate
    FP = ee.Number(Abs.filterMetadata('classification','greater_than',cutoff).size());
    FPR = FP.divide(Abs.size());
    # precision
    Precision = TP.divide(TP.add(FP));
    # sum of sensitivity and specificity
    SUMSS = TPR.add(TNR);
    return ee.Feature(None,{'cutoff': cutoff, 'TP':TP, 'TN':TN, 'FP':FP, 'FN':FN, 'TPR':TPR, 'TNR':TNR, 'FPR':FPR, 'Precision':Precision, 'SUMSS':SUMSS});

  return ee.FeatureCollection(seq.map(cut_));

# Calculate AUC_ROC
def getAUCROC(x):
  X = ee.Array(x.aggregate_array('FPR'));
  Y = ee.Array(x.aggregate_array('TPR')); 
  X1 = X.slice(0,1).subtract(X.slice(0,0,-1));
  Y1 = Y.slice(0,1).add(Y.slice(0,0,-1));
  return X1.multiply(Y1).multiply(0.5).reduce('sum',[0]).abs().toList().get(0);

def AUCROCaccuracy(x):
  HSM = ee.Image(images.get(x));
  TData = ee.FeatureCollection(TestingDatasets.get(x));
  Acc = getAcc(HSM, TData);
  return getAUCROC(Acc);

# Calculate AUC_PRG
def getAUCPR(roc):
  X = ee.Array(roc.aggregate_array('TPR'));
  Y = ee.Array(roc.aggregate_array('Precision')); 
  X1 = X.slice(0,1).subtract(X.slice(0,0,-1));
  Y1 = Y.slice(0,1).add(Y.slice(0,0,-1));
  return X1.multiply(Y1).multiply(0.5).reduce('sum',[0]).abs().toList().get(0);

def AUCPRaccuracy(x):
  HSM = ee.Image(images.get(x));
  TData = ee.FeatureCollection(TestingDatasets.get(x));
  Acc = getAcc(HSM, TData);
  return getAUCPR(Acc);

## Use 'folium' to visualize GEE maps

In [ ]:
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

## Model parameters and input

In [ ]:
all_assets = get_asset_list("users/hywong")
print('Total No. of assets in pool: {}'.format(len(all_assets)))

Total No. of assets in pool: 604


In [ ]:
## Model parameters 
target_case = 'occ_gcloud/' # gee asset folder name
outputFolder = 'checkEqual_R_GEE_brt_sub'

# set seed
Seed=78731

# Define partition for training and testing data
split = 0.80

# Define number of repetitions
numiter = 10

# pixel-size and scale for cross validation in grid
occ_limit= 4899
GrainSize = 900
Scale = 500000

# # subcontinental study area:
# subcont = ee.Geometry.BBox(-125,  25,-90, 43);

# global study area:
# some predictor variables are not covering all earth e.g. SRTM due to satellite instrument 

# global_aoi = ee.Geometry.BBox(-179.99999, -60, 179.99999, 60); # global area
SUB_AOI = ee.Geometry.BBox(-125, 25, -90, 43); # subcontinental area
# SUB_AOI = ee.Geometry.BBox(-125, 24, -65, 50); # conterminous USA
# SUB_AOI = ee.Geometry.BBox(72, 18, 136, 58); # China

AOI = SUB_AOI;

# iterate assets and grab the name of FeatureCollection:
target_asset = 'projects/earthengine-legacy/assets/users/hywong/'+target_case
assetList = [s for s in all_assets if target_asset in s]

# Environmental covariates in GEE data catalog
BIO = ee.Image("WORLDCLIM/V1/BIO");
Terrain = ee.Image("USGS/SRTMGL1_003");
predictors = BIO.addBands(Terrain);
watermask =  Terrain.select('elevation').gt(0);
predictors = predictors.updateMask(watermask).clip(AOI);
bands = ['bio02','bio03','bio07','bio08','bio09','bio10','bio14','bio15','bio16','bio18','bio19','elevation'];
predictors = predictors.select(bands);

# Set classifiers 
# "Random_Forest" Gradient_Tree_Boost
classifier_type = "Gradient_Tree_Boost"

# Set absence/background methods 
# pseudoAbs_k_means background
pseudoAbs_mode = "pseudoAbs_k_means"

In [ ]:
# check parameters and the assets are updated
print(classifier_type,pseudoAbs_mode,outputFolder,len(assetList))
assetList

In [ ]:
#### send to gee and run for each species 
for index, PlantName in enumerate(assetList):
  if index > 50:
    print(PlantName)
    Data = ee.FeatureCollection(PlantName)
    # AOI_zoom = Data.geometry()
    Data = RemoveDuplicates(Data)

    # a random reducer for computational timed out if needed
    Data = Data.randomColumn().sort('random').limit(ee.Number(occ_limit))
    
    # This will prevent having presence and pseudo-absences in the same pixel. 
    mask = Data.reduceToImage(['random'],ee.Reducer.first()).reproject('EPSG:4326', None, ee.Number(GrainSize)).mask().neq(1).selfMask();

    # PA points generation:
    # full background sample mode:
    if pseudoAbs_mode == 'background':
      AreaForPA = mask.updateMask(watermask).clip(AOI);
    elif pseudoAbs_mode == 'pseudoAbs_k_means':
      # pseudoAbs with k-means mode: 
      PixelVals = predictors.sampleRegions(collection= Data.randomColumn().sort('random').limit(200), properties= [], tileScale= 16, scale=GrainSize);
      # Perform k-means clusteringthe clusterer and train it using based on Eeuclidean distance.
      clusterer = ee.Clusterer.wekaKMeans(nClusters=2, distanceFunction="Euclidean").train(PixelVals);
      Clresult = predictors.cluster(clusterer);
      clustID = Clresult.sampleRegions(collection=Data.randomColumn().sort('random').limit(200), properties=[], tileScale=16, scale=GrainSize);
      clustID = ee.FeatureCollection(clustID).reduceColumns(ee.Reducer.mode(),['cluster']);
      clustID = ee.Number(clustID.get('mode')).subtract(1).abs();
      mask2 = Clresult.select(['cluster']).eq(clustID);
      AreaForPA = mask.updateMask(mask2).clip(AOI);
    else:
      print("need to specify pseudo absence method!")
      break

    # Create grid and remove cells outside AOI see front for Scale of spatial blocks
    grid = makeGrid(AOI, Scale);
    Grid = watermask.reduceRegions(collection=grid, reducer=ee.Reducer.mean()).filter(ee.Filter.neq('mean',None));
    # map to random numbers
    results = ee.List([35,68,43,54,17,46,76,88,24,12]).map(SDM);
    results = results.flatten();

    # Extract all model predictions
    images = ee.List.sequence(0,ee.Number(numiter).multiply(4).subtract(1),4).map(getResults);

    # Calculate mean of all individual model runs
    ModelAverage = ee.ImageCollection.fromImages(images).mean();
    
    #####################################################################################################
    
    # # Model accuracy metrics using testing data
    # TestingDatasets = ee.List.sequence(3,ee.Number(numiter).multiply(4).subtract(1),4).map(getResults);
    # AUCROCs = ee.List.sequence(0,ee.Number(numiter).subtract(1),1).map(AUCROCaccuracy);
    # AUCPRs = ee.List.sequence(0,ee.Number(numiter).subtract(1),1).map(AUCPRaccuracy);
    # def AUC_ROCs(element):
    #   return ee.Feature(None,{'AUCROC':element})
    # def AUC_PRs(element):
    #   return ee.Feature(None,{'AUCPR':element})

    # AUCROCs = ee.batch.Export.table.toDrive(collection=ee.FeatureCollection(AUCROCs.map(AUC_ROCs)), 
    #                 folder='nymphs_csv_BRT_outputs', 
    #                 description=PlantName.rsplit('/', 1)[-1]+'_AUCROC',
    #                 fileFormat='CSV') 
    # AUCROCs.start() 

    # AUCPRs = ee.batch.Export.table.toDrive(collection=ee.FeatureCollection(AUCPRs.map(AUC_PRs)), 
    #                 folder='nymphs_csv_BRT_outputs', 
    #                 description=PlantName.rsplit('/', 1)[-1]+'_AUCPR',
    #                 fileFormat='CSV') 
    # AUCPRs.start()

    ################################### export suitability maps ########################################

    averaged_Suitability = ee.batch.Export.image.toDrive(image=ModelAverage, 
                        folder=outputFolder, 
                        scale=GrainSize,
                        maxPixels=1e13,
                        region=AOI,
                        crs ='EPSG:4326',
                        description=PlantName.rsplit('/', 1)[-1])
    averaged_Suitability.start() 
    
    #####################################################################################################

In [ ]:
# print task conditions
while averaged_Suitability.active():
  print('Task (id: {}).'.format(averaged_Suitability.id))
  time.sleep(100)

## Visualize species presences within Colab

In [ ]:
geometry = ee.Geometry.Rectangle([120, 40, 120, 40])
center = geometry.centroid().getInfo()['coordinates']
center.reverse()
occ_limit_vis=500
Mapdisplay(center,{'data':RemoveDuplicates(ee.FeatureCollection('projects/earthengine-legacy/assets/users/hywong/occ_')).randomColumn().sort('random').limit(ee.Number(occ_limit_vis))
.getInfo()},zoom_start=2)
# Mapdisplay(center,{'AOI':AOI.getInfo()},zoom_start=7)

## Batch delete GEE assest

In [ ]:
# # to batch delete assets. Use with care:  
# assetList
# for index, asses in enumerate(assetList):
#   if index >= 0:
#     ee.data.deleteAsset(asses)	